# Multivariate analysis for CD8 and NK cell activation

Script related to figure 6b

### Aim:
- Perform multivariate T test for potential activation of CD8 and NK cells

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.4.0 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.4.6   
 [9] uuid_0.1-4      IRkernel_0.8.14 jsonlite_1.6.1  digest_0.6.25  
[13] repr_0.17       rlang_0.4.6     evaluate_0.14  

## Load packages

In [12]:
library(Hotelling)

Loading required package: corpcor


In [3]:
?Hotelling

In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/")

## Load sample labels

In [3]:
samples <- read.table('Data/5/NACT_SampleLabels.txt',
                      sep='\t',
                      header=T,
                      row.names='Well')

In [4]:
matched <- c(3368,371,9717,8674,2359,564,7765,6965,9347)

samples$Matched <- samples$Case_mRNA %in% matched

In [5]:
head(samples)

,Name,Case_mRNA,NACT,Site,Bounds,Matched
A01,A01_3368A_Liver,3368,Untreated,other,in,TRUE
A02,A02_3368B_Liver,3368,Treated,other,out,TRUE
A03,A03_0371A_R_diaphragm,371,Untreated,other,out,TRUE
A04,A04_0371B_R_diaphragm,371,Treated,other,out,TRUE
A05,A05_9717A_R_ovary,9717,Untreated,ovary,in,TRUE
A06,A06_9717B_R_ovary,9717,Treated,ovary,out,TRUE


## Load ConsensusTME NES

In [6]:
consensus_data <- read.table('Data/5/ConsensusTME_NACT_NES.txt',
                             sep='\t',
                             header=T,
                             row.names='Term')

consensus_data <- t(consensus_data)
head(consensus_data)

,B_cells,Cytotoxic_cells,Dendritic_cells,Endothelial,Eosinophils,Fibroblasts,Macrophages,Macrophages_M1,Macrophages_M2,Mast_cells,Monocytes,NK_cells,Neutrophils,Plasma_cells,T_cells_CD4,T_cells_CD8,T_cells_gamma_delta,T_regulatory_cells
A01,0.2544767,0.16012391,0.23412023,0.14187053,-0.104189950,0.09241945,0.2670112,0.1720899,0.1814478,0.02192616,0.2304878,-0.02041523,0.1091831,0.3268816,-0.1512603,-0.12458821,-0.12105553,-0.17131674
A02,0.1831832,0.08682826,0.24272549,-0.03106225,-0.001941604,0.09346663,0.2836625,0.1048446,0.1960520,-0.11581174,0.2591486,-0.02526795,0.2488612,0.4832443,-0.1722784,-0.09932701,-0.11002029,-0.17662104
A03,0.3284338,0.25078253,0.35828990,0.35977859,0.118276210,0.30425047,0.4091265,0.2950658,0.3210669,0.04103930,0.3858617,0.02020867,0.3013136,0.4849676,0.1548997,0.09899607,0.02099488,0.12494436
A04,0.2229200,0.46928297,0.24966760,0.38595430,0.292185918,0.49939131,0.2848222,0.2185944,0.3128521,0.30652579,0.3205545,0.17078484,0.4332402,0.5125873,0.0690361,0.08209151,0.06589801,0.07791841
A05,0.1584463,0.14999672,0.09665491,-0.03893381,-0.136141994,0.03414850,0.2307931,0.1273806,0.2043502,-0.15383368,0.2006747,-0.09731896,0.2647767,0.2534280,-0.2120999,-0.16972010,-0.18042408,-0.28792353
A06,0.2086076,0.28042125,0.21956787,0.29585502,-0.127744151,0.03049037,0.3885820,0.2812594,0.3980291,-0.07557768,0.2615250,-0.01597149,0.2808125,0.3320963,-0.1439186,-0.14446257,-0.12065637,-0.19520560


### Split matched and unmatched samples data

In [7]:
matched   <- consensus_data[which(samples$Matched == TRUE ),]

unmatched <- consensus_data[which(samples$Matched == FALSE),]

In [8]:
tail(matched)

,B_cells,Cytotoxic_cells,Dendritic_cells,Endothelial,Eosinophils,Fibroblasts,Macrophages,Macrophages_M1,Macrophages_M2,Mast_cells,Monocytes,NK_cells,Neutrophils,Plasma_cells,T_cells_CD4,T_cells_CD8,T_cells_gamma_delta,T_regulatory_cells
B02,0.2402263,0.04165139,0.1960428,0.13219214,-0.03672601,0.26149548,0.3296904,0.2648137,0.2625680,0.11898768,0.2465675,-0.114436017,0.2835240,0.3383852,-0.16099866,-0.10924458,-0.10747554,-0.29836380
B03,0.2835227,0.33674780,0.4122715,0.22910920,0.22940228,0.28277541,0.4041324,0.3839678,0.4119708,0.05383048,0.2921816,0.127573340,0.3570923,0.6466582,0.03783387,0.14070057,0.11995260,0.15591034
B04,0.2839035,0.29739252,0.4115423,0.32445657,0.23703105,0.16866140,0.4024553,0.3650535,0.4139615,0.11777950,0.3647248,0.091471933,0.3717044,0.5847199,0.09239342,0.12828607,0.07697151,0.18938231
B05,0.2040914,0.10670975,0.2725495,0.17986191,0.42209691,0.23263157,0.3496187,0.2867881,0.2484168,0.02069893,0.3050340,0.004476703,0.4952381,0.5587154,0.11541886,0.05464495,-0.01055403,0.20189863
B06,0.1358404,-0.06047101,0.1364558,-0.01334995,-0.06623356,-0.03450019,0.3311684,0.2358284,0.1790985,-0.01447437,0.2186006,-0.174316371,0.1910897,0.1703691,-0.20636331,-0.20507770,-0.20014783,-0.35279681
B07,0.2521700,0.18689107,0.2492469,0.20180648,0.02932837,0.16933787,0.3171786,0.1382779,0.2975472,-0.06294442,0.2755501,0.040335033,0.2423303,0.5589227,-0.07622513,-0.01944428,-0.08464659,-0.07293489


### Remove columns not needed

In [9]:
matched    <- subset(
    cbind(matched, samples[which(samples$Matched == TRUE),]),
          select=-c(Name,Case_mRNA,Site,Bounds,Matched)
)

unmatched  <- subset(
    cbind(unmatched, samples[which(samples$Matched == FALSE),]),
    select=-c(Name,Case_mRNA,Site,Bounds,Matched)
)

## Performed matched analysis

In [10]:
head(matched)

,B_cells,Cytotoxic_cells,Dendritic_cells,Endothelial,Eosinophils,Fibroblasts,Macrophages,Macrophages_M1,Macrophages_M2,Mast_cells,Monocytes,NK_cells,Neutrophils,Plasma_cells,T_cells_CD4,T_cells_CD8,T_cells_gamma_delta,T_regulatory_cells,NACT
A01,0.2544767,0.16012391,0.23412023,0.14187053,-0.104189950,0.09241945,0.2670112,0.1720899,0.1814478,0.02192616,0.2304878,-0.02041523,0.1091831,0.3268816,-0.1512603,-0.12458821,-0.12105553,-0.17131674,Untreated
A02,0.1831832,0.08682826,0.24272549,-0.03106225,-0.001941604,0.09346663,0.2836625,0.1048446,0.1960520,-0.11581174,0.2591486,-0.02526795,0.2488612,0.4832443,-0.1722784,-0.09932701,-0.11002029,-0.17662104,Treated
A03,0.3284338,0.25078253,0.35828990,0.35977859,0.118276210,0.30425047,0.4091265,0.2950658,0.3210669,0.04103930,0.3858617,0.02020867,0.3013136,0.4849676,0.1548997,0.09899607,0.02099488,0.12494436,Untreated
A04,0.2229200,0.46928297,0.24966760,0.38595430,0.292185918,0.49939131,0.2848222,0.2185944,0.3128521,0.30652579,0.3205545,0.17078484,0.4332402,0.5125873,0.0690361,0.08209151,0.06589801,0.07791841,Treated
A05,0.1584463,0.14999672,0.09665491,-0.03893381,-0.136141994,0.03414850,0.2307931,0.1273806,0.2043502,-0.15383368,0.2006747,-0.09731896,0.2647767,0.2534280,-0.2120999,-0.16972010,-0.18042408,-0.28792353,Untreated
A06,0.2086076,0.28042125,0.21956787,0.29585502,-0.127744151,0.03049037,0.3885820,0.2812594,0.3980291,-0.07557768,0.2615250,-0.01597149,0.2808125,0.3320963,-0.1439186,-0.14446257,-0.12065637,-0.19520560,Treated


In [13]:
cd8_matched_fit = hotelling.test(T_cells_CD8*Cytotoxic_cells~NACT, data = matched, perm = TRUE)

  |======================================================================| 100%

In [14]:
cd8_matched_fit

Test stat:  1.3546 
Numerator df:  2 
Denominator df:  15 
Permutation P-value:  0.2813 
Number of permutations : 10000 

In [15]:
pdf('Figures/main/6/panels/6b_cd8_cytotoxic_matched.pdf')

plot(cd8_matched_fit,
     freq=TRUE,
     breaks=30,
     col="#FFDD15",
     xlim=c(0,25)
    )

dev.off()

pdf 
  2

In [16]:
nk_matched_fit = hotelling.test(NK_cells*Cytotoxic_cells~NACT, data = matched, perm = TRUE)

  |======================================================================| 100%

In [17]:
nk_matched_fit

Test stat:  5.4825 
Numerator df:  2 
Denominator df:  15 
Permutation P-value:  0.0238 
Number of permutations : 10000 

In [18]:
pdf('Figures/main/6/panels/6b_nk_cytotoxic_matched.pdf')

plot(nk_matched_fit,
     freq=TRUE,
     breaks=30,
     col="#70B09D",
     xlim=c(0,25)
    )

dev.off()

pdf 
  2

## Performed unmatched analysis

In [19]:
head(unmatched)

,B_cells,Cytotoxic_cells,Dendritic_cells,Endothelial,Eosinophils,Fibroblasts,Macrophages,Macrophages_M1,Macrophages_M2,Mast_cells,Monocytes,NK_cells,Neutrophils,Plasma_cells,T_cells_CD4,T_cells_CD8,T_cells_gamma_delta,T_regulatory_cells,NACT
B08,0.2498001,0.1454555298,0.22403339,0.20216359,-0.152162396,-0.03651893,0.3484167,0.2567119,0.20275182,-0.01906826,0.23239805,-0.06910932,0.18625729,0.4655307,-0.16401822,-0.10329955,-0.106470526,-0.16558988,Untreated
B09,0.1933006,0.0100639843,0.16839998,0.08350335,0.050037526,0.14307024,0.2745897,0.1688296,0.17643015,-0.05040785,0.21282044,-0.15095151,0.25534965,0.5082110,-0.18604703,-0.16329057,-0.195857597,-0.18388259,Treated
B10,0.2696923,0.1626807887,0.33781042,0.36189287,0.315413070,0.23546584,0.4502848,0.2421547,0.43870926,0.36489011,0.38368471,-0.06494259,0.41151377,0.6145221,-0.06454531,0.05888267,0.002355267,-0.05396421,Untreated
B11,0.3080569,0.0433975608,0.24843782,0.27937426,0.154183957,0.26661771,0.2910163,0.2152640,0.21200374,0.10240685,0.26178467,-0.03505903,0.25641958,0.4577898,-0.08345257,-0.06288239,-0.079102146,-0.09249925,Treated
B12,0.1218557,-0.0101676989,-0.04453268,0.10808702,-0.314087264,0.05244197,0.1840254,-0.0043899,0.09819619,-0.31202210,0.06983559,-0.23581790,0.04428593,0.3212982,-0.31160618,-0.24535552,-0.305152598,-0.27984613,Untreated
C01,0.2401842,0.0009513548,0.15064419,0.23364429,-0.008568283,0.22263256,0.2327631,0.1524298,0.21435683,-0.08310467,0.19027735,-0.11658550,0.24027765,0.3913319,-0.22732101,-0.21474312,-0.188621872,-0.23188398,Treated


In [21]:
cd8_unmatched_fit = hotelling.test(T_cells_CD8*Cytotoxic_cells~NACT, data = unmatched, perm = TRUE)

  |======================================================================| 100%

In [22]:
cd8_unmatched_fit

Test stat:  0.46294 
Numerator df:  2 
Denominator df:  35 
Permutation P-value:  0.6383 
Number of permutations : 10000 

In [23]:
pdf('Figures/main/6/panels/6b_cd8_cytotoxic_unmatched.pdf')

plot(cd8_unmatched_fit,
     freq=TRUE,
     breaks=30,
     col="#FFDD15",
     xlim=c(0,25)
    )

dev.off()

pdf 
  2

In [25]:
nk_unmatched_fit = hotelling.test(NK_cells*Cytotoxic_cells~NACT, data = unmatched, perm = TRUE)

  |======================================================================| 100%

In [26]:
nk_unmatched_fit

Test stat:  0.34055 
Numerator df:  2 
Denominator df:  35 
Permutation P-value:  0.7166 
Number of permutations : 10000 

In [27]:
pdf('Figures/main/6/panels/6b_nk_cytotoxic_unmatched.pdf')

plot(nk_unmatched_fit,
     freq=TRUE,
     breaks=30,
     col="#70B09D",
     xlim=c(0,25)
    )

dev.off()

pdf 
  2

# End script